In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import qutip as qt 
from ECD_control.ECD_optimization.tf_adam_optimizer import AdamOptimizer
from ECD_control.gate_sets import ECDGateSet
from ECD_control.ECD_optimization.GateSynthesizer import GateSynthesizer
from ECD_control.ECD_optimization.optimization_analysis import OptimizationAnalysis, OptimizationSweepsAnalysis
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [ ]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
N = 40
# We initialize the ECDGateSet here
gate_set_params = {
    'N_cav' : N,
    'beta_scale' : 3.0, #maximum |beta| for random initialization  
}
ECD_gate_set = ECDGateSet(**gate_set_params)

In [ ]:
#The target oscillator state.
fock = 1
psi_t = qt.basis(N,fock) #target state

In [ ]:
#the optimization options
synth_params = {
    'gateset' : ECD_gate_set,
    'N_blocks':6,
    'N_multistart' : 100, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 100, #number of epochs before termination
    'epoch_size' : 10, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'beta_scale' : 3.0, #maximum |beta| for random initialization
    'initial_states' : [qt.tensor(qt.basis(2,0),qt.basis(N,0))], #qubit tensor oscillator, start in |g> |0>
    'target_states' : [qt.tensor(qt.basis(2,1), psi_t)], #end in |e> |target>.
    'name' : 'Fock %d' % fock, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}
gatesynth = GateSynthesizer(**synth_params)

In [ ]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.print_info()

In [ ]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
opt.optimize()

In [ ]:
#can print info, including the best circuit found.
gatesynth.print_info() 

In [ ]:
print(gatesynth.best_fidelity())

In [ ]:
#can also get the best circuit parameters directly, could save this to a .npz file.
best_circuit =  gatesynth.best_circuit()
betas = best_circuit['betas']
phis = best_circuit['phis']
thetas = best_circuit['thetas']
print(best_circuit)